In [1]:
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import os
from os.path import join as pjoin
import numpy as np
import pandas as pd
from time import time, sleep, strftime, localtime
from datetime import datetime

%reload_ext autoreload
%autoreload 1

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 100)

display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [2]:
import re, string
import requests
from bs4 import BeautifulSoup
from itertools import groupby


In [13]:
main_url = 'http://www.azlyrics.com/'
band_name = "metallica"
band_url = band_name[0]+'/'+band_name+'.html'
# band_url = 'm/metallica.html'
album_list_url = main_url + band_url
print(album_list_url)
page = requests.get(album_list_url).text
soup = BeautifulSoup(page, "html.parser")

http://www.azlyrics.com/m/metallica.html


In [10]:
# with open("azlyrics_ironmaiden.html", "w") as file:
#     file.write(str(soup))

http://www.azlyrics.com/m/metallica.html


In [4]:
# file = open("azlyrics_ironmaiden.html")
# soup = BeautifulSoup(file, "html.parser")

---------

In [14]:
list_albums = soup.find_all(attrs={"id":"listAlbum"})[0]
albums_and_songs = [el for el in list_albums if el not in ["\n", "<br/>"] ][:-1]
albums = [el for el in albums_and_songs if el.attrs["class"][0] == "album"]
songs = [list(g) for k, g in groupby(albums_and_songs, key=lambda x: x.attrs["class"][0] != "album") if k]
# alb_song_map = {}
# for i in album_and_songs:
#     if 'album'==i.attrs["class"][0]:
full_map = {
    "title":[],
    "year":[],
    "is_single":[],
    "song_names":[],
    "song_hrefs":[]
}
for alb, song_list in zip(albums, songs):
    full_album_name = alb.text
    is_single = False
    if "album:" in full_album_name:
        album_name = re.sub("[^a-zA-Z0-9 ]", "", full_album_name[7:])[:-4]
        year = int(full_album_name[-5:-1])
    elif "single:" in full_album_name:
        album_name = re.sub("[^a-zA-Z0-9 ]", "", full_album_name[6:])[:-4]
        year = int(full_album_name[-5:-1])
        is_single = True
    else:
        album_name = re.sub("[^a-zA-Z0-9 ]", "", full_album_name)
        year = -1
        
    song_names = [[i.text for i in s.find_all('a')][0] for s in song_list]
    song_hrefs = [[pjoin(main_url, i['href'][3:]) for i in s.find_all('a')][0] for s in song_list]

    full_map["title"].append(album_name)
    full_map["year"].append(year)
    full_map["is_single"].append(is_single)
    full_map["song_names"].append(song_names)
    full_map["song_hrefs"].append(song_hrefs)
    
df_band_albums = pd.DataFrame(full_map)

ValueError: invalid literal for int() with base 10: 'Reed'

In [15]:
df_band_albums.head()

,title,year,is_single,song_names,song_hrefs
0,Iron Maiden,1980,False,"[Prowler, Sanctuary, Remember Tomorrow, Running Free, Phantom Of The Opera, Strange World, Charl...","[http://www.azlyrics.com/lyrics/ironmaiden/prowler.html, http://www.azlyrics.com/lyrics/ironmaid..."
1,Killers,1981,False,"[Wrathchild, Murders In The Rue Morgue, Another Life, Innocent Exile, Killers, Prodigal Son, Pur...","[http://www.azlyrics.com/lyrics/ironmaiden/wrathchild.html, http://www.azlyrics.com/lyrics/ironm..."
2,The Number Of The Beast,1982,False,"[Invaders, Children Of The Damned, The Prisoner, 22 Acacia Avenue, The Number Of The Beast, Run ...","[http://www.azlyrics.com/lyrics/ironmaiden/invaders.html, http://www.azlyrics.com/lyrics/ironmai..."
3,Piece Of Mind,1983,False,"[Where Eagles Dare, Revelations, Flight Of Icarus, Die With Your Boots On, The Trooper, Still Li...","[http://www.azlyrics.com/lyrics/ironmaiden/whereeaglesdare.html, http://www.azlyrics.com/lyrics/..."
4,Powerslave,1984,False,"[Aces High, 2 Minutes To Midnight, Flash Of The Blade, The Duellists, Back In The Village, Power...","[http://www.azlyrics.com/lyrics/ironmaiden/aceshigh.html, http://www.azlyrics.com/lyrics/ironmai..."


In [121]:
# df_band_albums.to_csv("./df_band_albums.csv", index=False)

In [5]:
# df_band_albums = pd.read_csv("./df_band_albums.csv")
# df_band_albums["song_names"] = df_band_albums["song_names"].apply(lambda x: x.strip('][').split(', '))
# df_band_albums["song_hrefs"] = df_band_albums["song_hrefs"].apply(lambda x: x.strip('][').split(', '))
# df_band_albums.head()

,title,year,is_single,song_names,song_hrefs
0,Iron Maiden,1980,False,"['Prowler', 'Sanctuary', 'Remember Tomorrow', 'Running Free', 'Phantom Of The Opera', 'Strange W...","['http://www.azlyrics.com/lyrics/ironmaiden/prowler.html', 'http://www.azlyrics.com/lyrics/ironm..."
1,Killers,1981,False,"['Wrathchild', 'Murders In The Rue Morgue', 'Another Life', 'Innocent Exile', 'Killers', 'Prodig...","['http://www.azlyrics.com/lyrics/ironmaiden/wrathchild.html', 'http://www.azlyrics.com/lyrics/ir..."
2,The Number Of The Beast,1982,False,"['Invaders', 'Children Of The Damned', 'The Prisoner', '22 Acacia Avenue', 'The Number Of The Be...","['http://www.azlyrics.com/lyrics/ironmaiden/invaders.html', 'http://www.azlyrics.com/lyrics/iron..."
3,Piece Of Mind,1983,False,"['Where Eagles Dare', 'Revelations', 'Flight Of Icarus', 'Die With Your Boots On', 'The Trooper'...","['http://www.azlyrics.com/lyrics/ironmaiden/whereeaglesdare.html', 'http://www.azlyrics.com/lyri..."
4,Powerslave,1984,False,"['Aces High', '2 Minutes To Midnight', 'Flash Of The Blade', 'The Duellists', 'Back In The Villa...","['http://www.azlyrics.com/lyrics/ironmaiden/aceshigh.html', 'http://www.azlyrics.com/lyrics/iron..."
5,Somewhere In Time,1986,False,"['Caught Somewhere In Time', 'Wasted Years', 'Sea Of Madness', 'Heaven Can Wait', 'The Lonelines...","['http://www.azlyrics.com/lyrics/ironmaiden/caughtsomewhereintime.html', 'http://www.azlyrics.co..."
6,Seventh Son Of A Seventh Son,1988,False,"['Moonchild', 'Infinite Dreams', 'Can I Play With Madness?', 'The Evil That Men Do', 'Seventh So...","['http://www.azlyrics.com/lyrics/ironmaiden/moonchild.html', 'http://www.azlyrics.com/lyrics/iro..."
7,No Prayer For The Dying,1990,False,"['Tailgunner', 'Holy Smoke', 'No Prayer For The Dying', 'Public Enema Number One', 'Fates Warnin...","['http://www.azlyrics.com/lyrics/ironmaiden/tailgunner.html', 'http://www.azlyrics.com/lyrics/ir..."
8,Fear Of The Dark,1992,False,"['Be Quick Or Be Dead', 'From Here To Eternity', 'Afraid To Shoot Strangers', 'Fear Is The Key',...","['http://www.azlyrics.com/lyrics/ironmaiden/bequickorbedead.html', 'http://www.azlyrics.com/lyri..."
9,From Here To Eternity,1992,True,"['From Here To Eternity', ""I Can't See My Feelings"", 'Roll Over Vic Vella', 'No Prayer For The D...","['http://www.azlyrics.com/lyrics/ironmaiden/fromheretoeternity.html', 'http://www.azlyrics.com/l..."


In [6]:
albums = list([])
all_song_maps = {
    "album_name":[],
    "song_name":[],
    "song_lyrics":[]
}
albums_collected = [
    "Iron Maiden ",
    "Killers ",
    "The Number Of The Beast ",
    "Piece Of Mind ",
    "Powerslave ",
    "Somewhere In Time ",
    "Seventh Son Of A Seventh Son ",
    "No Prayer For The Dying ",
#     "Fear Of The Dark "
]
for album in df_band_albums["title"].unique():
    if album in albums_collected:
        continue
    print()
    q = "title==\""+album+"\""
    print(q)
    song_names = df_band_albums.query(q).loc[:,"song_names"].values[0] 
    song_hrefs = df_band_albums.query(q).loc[:,"song_hrefs"].values[0]
    for song_name, href in zip(song_names, song_hrefs):
        sleep(1)
        song_href = href.replace("\\","/")[1:-1]
        print(song_href)
        song_page = requests.get(song_href).text
        song_soup = BeautifulSoup(song_page, "html.parser")
        lyrics_section = song_soup.find("div", attrs={"class":"col-xs-12 col-lg-8 text-center"})
        song = lyrics_section.find("div", attrs={"class":""}).text
        
        all_song_maps["album_name"].append(album)
        all_song_maps["song_name"].append(song_name)
        all_song_maps["song_lyrics"].append(song.strip())
        sleep(np.random.randint(low=5, high=10))
        
    sleep(np.random.randint(low=30, high=60))


title=="Fear Of The Dark "
http://www.azlyrics.com/lyrics/ironmaiden/bequickorbedead.html
http://www.azlyrics.com/lyrics/ironmaiden/fromheretoeternity.html
http://www.azlyrics.com/lyrics/ironmaiden/afraidtoshootstrangers.html
http://www.azlyrics.com/lyrics/ironmaiden/fearisthekey.html
http://www.azlyrics.com/lyrics/ironmaiden/childhoodsend.html
http://www.azlyrics.com/lyrics/ironmaiden/wastinglove.html
http://www.azlyrics.com/lyrics/ironmaiden/thefugitive.html
http://www.azlyrics.com/lyrics/ironmaiden/chainsofmisery.html
http://www.azlyrics.com/lyrics/ironmaiden/theapparition.html
http://www.azlyrics.com/lyrics/ironmaiden/judasbemyguide.html
http://www.azlyrics.com/lyrics/ironmaiden/weekendwarrior.html
http://www.azlyrics.com/lyrics/ironmaiden/fearofthedark.html

title==" From Here To Eternity "
http://www.azlyrics.com/lyrics/ironmaiden/fromheretoeternity.html
http://www.azlyrics.com/lyrics/ironmaiden/icantseemyfeelings.html
http://www.azlyrics.com/lyrics/ironmaiden/rollovervicvella.h

In [9]:
pd.DataFrame(all_song_maps).to_csv("./"+band_name+".csv")